In [13]:
# 연습문제 : 청와대 국민 신문고 데이터 수집하기

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  # pip install numpy 를 해야 사용가능함
import pandas as pd  # pip install pandas 해야 사용가능하고 numpy 가 먼저 설치되어 있어야 함
import xlwt

import random
import os   # 폴더 생성하기 위해 필요함

#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print(" 연습문제: 청와대 국민 신문고 게시판 크롤링하기")
print("=" *80)

query_txt = '청와대국민신문고'

cnt = int(input("   1.크롤링 할 건수는 몇건입니까?: "))
f_dir = input("   2.결과 파일을 저장할 폴더명만 쓰세요(예:c:\\temp\\):")

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://www.epeople.go.kr/jsp/user/pp/UPpProposOpenList.paid?flag=A')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택


print("\n")
page_cnt = math.ceil(cnt / 10)
print("크롤링 할 총 페이지 번호: ",page_cnt)
print("=" *80)

# Step 4.  게시글 요약 정보 출력

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

click_cnt = 2   # 클릭한 페이지 번호 지정
no = 1
cno2=[]
title2=[]
part2=[]
rdate2=[]
status2=[]
view2=[]

for x in range(1, page_cnt+1) :
        
        print("%s 페이지의 요약 내용 수집 시작합니다 =======================" %x)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        result = soup.select('div.listForm1 > table > tbody > tr')
        
        for i in range(0, len(result) ):  
                
            print("\n")
            print("%s 번째 국민 신문고 게시글 정보입니다==========================" %no)
            
            f = open(ff_name, 'a',encoding='UTF-8')
            
            # 청원 등록 정보 수집
            content = result[i].find_all('td')
            
            cno = content[0].get_text( )       
            print('1.청원글번호: ',cno ,'\n')
            f.write('\n')
            f.write("%s 번째 국민 신문고 게시글 정보입니다==========================" %no + '\n')
            f.write('1.청원번호:' + cno + '\n')
            cno2.append(cno)
            
            # 청원제목 추출
            title = content[1].get_text( ).replace("\n","").strip()
            print('2.청원제목: ' , title , '\n')
            f.write('2.청원제목:' + title + '\n')
            title2.append(title)
            
            # 처리기관명 추출
            part = content[2].get_text( ).replace("\n","").strip()
            print('3.처리기관명: ' , part , '\n')
            f.write('3.처리기관명:' + part + '\n')
            part2.append(part)
            
            # 신청일 추출
            rdate = content[3].get_text( ).replace("\n","").strip()
            print('4.신청일: ' , rdate , '\n')
            f.write('4.신청일:' + rdate + '\n')
            rdate2.append(rdate)
            
            # 추진상황 추출
            status = content[4].get_text( ).replace("\n","").strip()
            print('5.추진상황: ' , status , '\n')
            f.write('5.추진상황:' + status + '\n')
            status2.append(status)
            
            # 조회수 추출
            view = content[5].get_text( ).replace("\n","").strip()
            print('6.조회수: ' , view , '\n')
            f.write('6.조회수:' + view + '\n')
            view2.append(view)
            
            f.close( )
            
            no += 1
            
            if no > cnt :
                break
                
        if click_cnt > page_cnt :
                break
        else :
                driver.find_element_by_link_text("""%s""" %click_cnt).click() # 다음 페이지번호 클릭
                click_cnt += 1
            
                time.sleep(2)

# Step 5. 출력 결과를 저장합니다
# 출력 결과를 표(데이터 프레임) 형태로 만들기

sin = pd.DataFrame()

sin['글번호'] = cno2
sin['글제목'] = pd.Series(title2) 
sin['처리기관명'] = pd.Series(part2)
sin['신청일'] = pd.Series(rdate2)
sin['추진상황'] = pd.Series(status2)
sin['조회수'] = pd.Series(view2)
            
# csv 형태로 저장하기
sin.to_csv(fc_name, encoding="utf-8-sig" , index=False)

# 엑셀 형태로 저장하기
sin.to_excel(fx_name,index=False)

#Step 6. 요약 정보 출력하기
e_time = time.time( )     
t_time = e_time - s_time
print("\n") 
print("=" *100)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *100)

 연습문제 6-3.  청와대 국민 신문고 게시판 크롤링하기
   1.크롤링 할 건수는 몇건입니까?: 35
   2.결과 파일을 저장할 폴더명만 쓰세요(예:c:\temp\):c:\data\


크롤링 할 총 페이지 번호:  4
1 페이지의 요약 내용 수집 시작합니다 =======================


1 번째 국민 신문고 게시글 정보입니다==========================
1.청원글번호:  11092 

2.청원제목:  하나은행.카드연체 

3.처리기관명:   

4.신청일:  2019-04-19 

5.추진상황:  민원이관 

6.조회수:  2 



2 번째 국민 신문고 게시글 정보입니다==========================
1.청원글번호:  11091 

2.청원제목:  압류 방지 통장 행복지킴이 통장 입금 가능 대상 급여 확대 

3.처리기관명:  보건복지부 

4.신청일:  2019-04-19 

5.추진상황:  제안심사 

6.조회수:  1 



3 번째 국민 신문고 게시글 정보입니다==========================
1.청원글번호:  11090 

2.청원제목:  자연환경지킴이 자격증 발급제도 도입 

3.처리기관명:  환경부 

4.신청일:  2019-04-19 

5.추진상황:  제안심사 

6.조회수:  1 



4 번째 국민 신문고 게시글 정보입니다==========================
1.청원글번호:  11089 

2.청원제목:  조현병(정신분열증)환자 특별관리 방안 

3.처리기관명:  보건복지부 

4.신청일:  2019-04-19 

5.추진상황:  제안심사 

6.조회수:  1 



5 번째 국민 신문고 게시글 정보입니다==========================
1.청원글번호:  11088 

2.청원제목:  비닐,스티로폼 제품의 폐기물 처리방안 

3.처리기관명:  환경부 

4.신청일:  2019-04-19 

5.추진상황:  제안심사 

6.조회수:  2 



